In [1]:
import warnings
# warnings.filterwarnings("ignore", category=FutureWarning) 

def warn(*args, **kwargs):
    pass

warnings.warn = warn

In [2]:
import os
import pandas as pd
import numpy as np

#import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
color = sns.color_palette()

%matplotlib inline

In [ ]:
# Autoregressive Integrated Moving Average with boundaries
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic

In [4]:
test = pd.read_csv('./csv/test.csv')
test['dates'] = pd.to_datetime(test.dates)

In [5]:
fin_sku_df = pd.DataFrame()

for file_name in os.listdir('./csv'):
    
    if file_name.endswith('_group_sku.csv'):
        
        gr_sku = pd.read_csv("./csv/"+file_name)
        gr_sku['dates'] = pd.to_datetime(gr_sku.dates)

        gr_sku['price_per_sku'] = gr_sku.groupby('SKU').price_per_sku.ffill()
        
        fin_sku_df = pd.concat((fin_sku_df, gr_sku), axis=0)

In [6]:
sku_arr = fin_sku_df.SKU.unique()

In [7]:
fin_sku_pred_df = pd.DataFrame() 

for _sku in tqdm(sku_arr, total=len(sku_arr)): 
    
    train_df = fin_sku_df[fin_sku_df.SKU == _sku].sort_values('dates')
    test_df = test[test.SKU == _sku].sort_values('dates')

    ARIMA_ = ARIMA(train_df.price_per_sku.values, order=(30,1,20))
    # ARIMA_ = ARIMA(log_returns, order=(20,1,20))
    ARIMA_model = ARIMA_.fit() #start_ar_lags = 1
#     print(ARIMA_model.summary())
    test_df['price_per_sku'] = ARIMA_model.predict(start = len(train_df), 
                                                   end = len(train_df) + len(test_df)-1)
    fin_sku_pred_df = pd.concat((fin_sku_pred_df, test_df), axis=0)
    

100%|██████████| 25/25 [07:12<00:00, 17.28s/it]


In [8]:
fin_sku_pred_df

,dates,SKU,price_per_sku
21,2019-04-10,32950,449.356154
22,2019-04-11,32950,448.148569
23,2019-04-12,32950,446.069129
24,2019-04-13,32950,447.369724
25,2019-04-16,32950,452.685453
...,...,...,...
101,2019-12-26,89272,125.664045
112,2019-12-27,89272,125.644273
134,2019-12-28,89272,126.678554
157,2019-12-30,89272,126.333769


In [9]:
fin_sku_pred_df.to_csv('./results/predictions.csv', index=False)